In [1]:
#!/usr/bin/env python
# -*- encoding: utf-8 -*-
#-------------------------------------------------------------------------------
'''
@Author  :   {SEASON}
@License :   (C) Copyright 2013-2022, {OLD_IT_WANG}
@Contact :   {shiter@live.cn}
@Software:   PyCharm
@File    :   NLP_DEMO -- ThemeRiver
@Time    :   2020/9/13 23:14
@Desc    :

'''
#-------------------------------------------------------------------------------


import re
import jieba
import pandas as pd
import openpyxl
import pyecharts.options as opts
from pyecharts.charts import ThemeRiver
import pandas as pd


import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

sys.path

['D:\\ProgramData\\Anaconda3\\envs\\NLP_DEMO\\python36.zip',
 'D:\\ProgramData\\Anaconda3\\envs\\NLP_DEMO\\DLLs',
 'D:\\ProgramData\\Anaconda3\\envs\\NLP_DEMO\\lib',
 'D:\\ProgramData\\Anaconda3\\envs\\NLP_DEMO',
 '',
 'C:\\Users\\Lenovo\\AppData\\Roaming\\Python\\Python36\\site-packages',
 'D:\\ProgramData\\Anaconda3\\envs\\NLP_DEMO\\lib\\site-packages',
 'D:\\ProgramData\\Anaconda3\\envs\\NLP_DEMO\\lib\\site-packages\\win32',
 'D:\\ProgramData\\Anaconda3\\envs\\NLP_DEMO\\lib\\site-packages\\win32\\lib',
 'D:\\ProgramData\\Anaconda3\\envs\\NLP_DEMO\\lib\\site-packages\\Pythonwin',
 'D:\\ProgramData\\Anaconda3\\envs\\NLP_DEMO\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Lenovo\\.ipython',
 'D:\\code\\python\\csdn_nlp\\NLP_DEMO']

In [2]:
%load_ext autoreload
%autoreload 2


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# set up display area to show dataframe in jupyter qtconsole

import matplotlib
%matplotlib inline

#显示列的条数
pd.set_option('max_columns',1000) 
#显示行的条数
pd.set_option('max_row',300) 
#显示数值的精度
pd.set_option('display.float_format', lambda x: '%.5f' % x)

#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)



In [3]:
from Database import blog

In [4]:
from Database import blog, mySQLiteForblog
from src import assistance_tool



STR_PATH_SQLITE = 'sqlite:///../../Database/NLP_demo.db?check_same_thread=False'

DBSession = mySQLiteForblog.get_conn(STR_PATH_SQLITE, True)

blog = blog.CsdnBlog


all_blog_label = DBSession.query(blog.label).all()

# print(all_blog_label)

df = pd.DataFrame(DBSession.query(blog).all())


# print(all_blog_label)

2020-09-27 21:05:23,322 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-09-27 21:05:23,322 INFO sqlalchemy.engine.base.Engine ()
2020-09-27 21:05:23,332 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-09-27 21:05:23,332 INFO sqlalchemy.engine.base.Engine ()
2020-09-27 21:05:23,336 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-09-27 21:05:23,338 INFO sqlalchemy.engine.base.Engine SELECT "CSDN_Blog".label AS "CSDN_Blog_label" 
FROM "CSDN_Blog"
2020-09-27 21:05:23,339 INFO sqlalchemy.engine.base.Engine ()
2020-09-27 21:05:23,615 INFO sqlalchemy.engine.base.Engine SELECT "CSDN_Blog".article_id AS "CSDN_Blog_article_id", "CSDN_Blog".title AS "CSDN_Blog_title", "CSDN_Blog".content AS "CSDN_Blog_content", "CSDN_Blog".create_time AS "CSDN_Blog_create_time", "CSDN_Blog".click_number AS "CSDN_Blog_click_number", "CSDN_Blog".comment_number AS "CSDN_Blog_comment_number", "CSDN_Blog".l

In [5]:
df = pd.read_sql(sql = DBSession.query(blog)\
                         .with_entities(blog.label,
                                        blog.title).statement, 
                 con = DBSession.bind)

2020-09-27 21:05:23,820 INFO sqlalchemy.engine.base.OptionEngine SELECT "CSDN_Blog".label, "CSDN_Blog".title 
FROM "CSDN_Blog"
2020-09-27 21:05:23,821 INFO sqlalchemy.engine.base.OptionEngine ()


In [6]:
df

,label,title
0,",\n 生活感悟 ,\n ...",从前有一个程序员，成天写代码，后来，他屎了。。。
1,",\n 自然语言处理实战入门",《自然语言处理实战入门》基础知识 ---- NLP开源工具包与云服务提供商
2,",\n 老王和他的IT界朋友们",PCA主成分分析（上）
3,",\n 自然语言处理实战入门",《自然语言处理实战入门》基础知识 ----机器学习与深度学习组件
4,",\n 自然语言处理实战入门",《自然语言处理实战入门》 深度学习组件TensorFlow2.0---- 初探
...,...,...
403,",centos database ...",Centos学习笔记---文件搜索命令
404,",",计算机专业毕业生，求职升学，英语自我介绍
405,",\n c++ ,\n ...","VC之CString,wchar_t,int,string,char*之间的转换"
406,",\n Linux功能实现 ,centos ...",centos 学习笔记--文件处理命令


In [62]:
str_all = ''
str_all_list = []
count=0
for tup in df.itertuples():
    # 获取每篇文章的 标签列表
    str_temp_list = tup[1].split(',')
    str_temp_list1 = list(x.strip() for x in str_temp_list)
    
    
    # 单篇文章的标签列表进行按照空格分割拆分
    for label in str_temp_list1:
        str_temp_list2 = label.strip().split(' ')
        str_all_list = str_all_list + str_temp_list2
  

str_all_list_new = list(x for x in str_all_list if x!='')


In [66]:
str_all_list_new

dic = {}
for key in str_all_list_new:
    dic[key] = dic.get(key,str_all_list_new.count(key))
print(len(dic))

209


In [68]:
sorted(dic.items(), key=lambda item:item[1], reverse=True)

[('c++', 162),
 ('leetcode', 131),
 ('python', 76),
 ('算法设计与分析', 70),
 ('OpenCV', 67),
 ('计算机视觉', 63),
 ('图形图像', 62),
 ('algorithm', 50),
 ('计算机视觉OpenCV实现', 46),
 ('python实现', 42),
 ('老王和他的IT界朋友们', 33),
 ('机器学习', 28),
 ('程序人生', 25),
 ('windows功能实现', 25),
 ('大数据', 24),
 ('自然语言处理实战入门', 19),
 ('大数据ETL实践探索', 19),
 ('生活感悟', 18),
 ('自然语言处理', 17),
 ('NLP', 16),
 ('特征点', 16),
 ('笔试题', 16),
 ('opencv', 15),
 ('简单NLP分析套路', 14),
 ('代码片', 14),
 ('null', 14),
 ('c', 13),
 ('面试题', 12),
 ('windows', 12),
 ('string', 12),
 ('立体视觉', 11),
 ('Graph', 11),
 ('Cuts', 11),
 ('图像分割', 11),
 ('系统运维', 10),
 ('图论及其算法在计算机视觉中的应用', 10),
 ('二叉树', 10),
 ('STL', 10),
 ('unix', 10),
 ('并行计算', 9),
 ('Linux功能实现', 9),
 ('linux', 9),
 ('脚本', 9),
 ('汉语分词', 8),
 ('数据清洗', 8),
 ('web开发', 8),
 ('OpenGL', 8),
 ('立体匹配', 8),
 ('链表', 8),
 ('数据库', 8),
 ('system', 8),
 ('centos', 8),
 ('shell', 8),
 ('OpenCV特征点检测', 7),
 ('openmp', 7),
 ('stereo', 7),
 ('matching', 7),
 ('Django', 7),
 ('OpenMP', 6),
 ('京东', 6),
 ('跨平台通信系统', 6),
 ('fi